In [1]:
from importlib import reload

In [2]:
import xarray as xr
from dominosee.eca import (
    get_eca_precursor_from_events,
    get_eca_precursor_confidence
)
from dominosee.utils.blocking import process_eca_blockwise, combine_blocks

In [ ]:
from numba import njit, guvectorize, prange
from dask.distributed import Client, performance_report
client = Client(processes=False)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.245.92.223:8787/status,
Dashboard: http://10.245.92.223:8787/status,Workers: 1
Total threads: 48,Total memory: 251.39 GiB
Status: running,Using processes: False
Comm: inproc://10.245.92.223/1404053/1,Workers: 1
Dashboard: http://10.245.92.223:8787/status,Total threads: 48
Started: Just now,Total memory: 251.39 GiB
Comm: inproc://10.245.92.223/1404053/4,Total threads: 48
Dashboard: http://10.245.92.223:35367/status,Memory: 251.39 GiB
Nanny: None,


In [13]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", 
                                                                                             "longitude": "lon", 
                                                                                             "spi30d": "SPI1"}).chunk({"time": -1, 
                                                                                                                       "lat": 50,
                                                                                                                       "lon": 50})
ds.SPI1

<xarray.DataArray 'SPI1' (time: 10957, lat: 203, lon: 250)> Size: 2GB
dask.array<xarray-SPI1, shape=(10957, 203, 250), dtype=float32, chunksize=(10957, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 2kB 73.0 73.25 73.5 73.75 ... 134.5 134.8 135.0 135.2
  * lat      (lat) float64 2kB 54.25 54.0 53.75 53.5 53.25 ... 4.5 4.25 4.0 3.75
Attributes:
    units:               1
    calibration_period:  
    freq:                D
    window:              30
    scipy_dist:          gamma
    method:              PWM
    group:               time.dayofyear
    time_indexer:        {}

In [14]:
event_a = ds.SPI1 < -1.5
event_b = event_a.copy()

In [17]:
block_size = 200  # Adjust based on your memory constraints
output_dir = "eca_blocks"

process_eca_blockwise(
    get_eca_precursor_from_events,
    event_a,
    event_b,
    output_dir=output_dir,
    block_size=block_size,
    delt=2,
    sym=True,
    tau=0
)

Processing blocks:   0%|          | 0/2 [00:00<?, ?it/s]/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 20
  tmp = blockwise(
Processing blocks: 100%|██████████| 2/2 [06:00<00:00, 180.17s/it]


In [18]:
dc = xr.open_mfdataset("eca_blocks/eca_block_*.nc")
dc

<xarray.Dataset> Size: 5GB
Dimensions:  (latA: 203, latB: 203, lonA: 250, lonB: 250)
Coordinates:
  * lonA     (lonA) float64 2kB 73.0 73.25 73.5 73.75 ... 134.8 135.0 135.2
  * latA     (latA) float64 2kB 54.25 54.0 53.75 53.5 ... 4.5 4.25 4.0 3.75
  * lonB     (lonB) float64 2kB 73.0 73.25 73.5 73.75 ... 134.8 135.0 135.2
  * latB     (latB) float64 2kB 54.25 54.0 53.75 53.5 ... 4.5 4.25 4.0 3.75
Data variables:
    SPI1     (latA, latB, lonA, lonB) uint16 5GB dask.array<chunksize=(200, 200, 250, 250), meta=np.ndarray>

In [19]:
dc.close()